# Training

In [10]:
# decision tree? robust with noise (especially if pruned), can handle irrelevant data
# Naive bayes? not too good because of independence assumption
# SVM? widely used, need to find the best kernel
# nearest neighbors? data must be scaled, not too good with irrelevant features
# neural net? requires a lot of time and a lot of data, can deal with irrelevant features, can overfit, local minima issues
# ensemble?

# decision tree, svm, nearest neigh, ensemble

In [16]:
# get clean data
import os
import sys
import warnings
warnings.simplefilter("ignore")
import pandas as pd
import numpy as np
import sklearn as sk
import sklearn.naive_bayes
import sklearn.model_selection
import sklearn.tree
from sklearn.model_selection import GridSearchCV
import sklearn.preprocessing
import sklearn.pipeline
import sklearn.decomposition
import sklearn.neighbors
import sklearn.svm
import sklearn.ensemble
import matplotlib.pyplot as plt
import time

# need to pip install import_ipynb
import import_ipynb
from data_preperation import data

# at this point, data should be clean 
print(data)


0


# Feature Engineering 

In [17]:
# maybe do PCA?

# Training

# Testing 

# Predicting